In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.0,
  base_url="https://openai.vocareum.com/v1"
)

In [3]:
logs = []
parser = StrOutputParser()
parse_and_log_chain = RunnableParallel(
  output=parser,
  log=RunnableLambda(lambda x: logs.append(x))
)

In [6]:
# TODO - Your prompt Template
idea_prompt = PromptTemplate(
  template=(
    "You're a creative business advisor. "
    "Generate one innovative business idea in the industry: "
    "{industry}."
    "Provide a brief description of the ideia."
  )
)

In [8]:
idea_chain = idea_prompt | llm |parse_and_log_chain

In [9]:
idea_result = idea_chain.invoke("Agro")

In [10]:
idea_result["output"]

'**Business Idea: Smart Vertical Farming Pods (SVFP)**\n\n**Description:**\nSmart Vertical Farming Pods (SVFP) are modular, self-sustaining farming units designed for urban environments, utilizing advanced hydroponics and IoT technology. Each pod is a compact, stackable unit that can be installed in backyards, rooftops, or even inside homes, allowing individuals and small businesses to grow fresh produce year-round, regardless of climate conditions.\n\nThe SVFP features an integrated smart system that monitors and optimizes growth conditions, including light, temperature, humidity, and nutrient levels, using AI algorithms. Users can control and monitor their pods through a mobile app, receiving real-time updates and tips for maximizing yield. The pods are equipped with solar panels to ensure energy efficiency and sustainability.\n\nAdditionally, the business model includes a subscription service for nutrient packs and seeds, as well as an online community platform where users can share

In [11]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods (SVFP)**\n\n**Description:**\nSmart Vertical Farming Pods (SVFP) are modular, self-sustaining farming units designed for urban environments, utilizing advanced hydroponics and IoT technology. Each pod is a compact, stackable unit that can be installed in backyards, rooftops, or even inside homes, allowing individuals and small businesses to grow fresh produce year-round, regardless of climate conditions.\n\nThe SVFP features an integrated smart system that monitors and optimizes growth conditions, including light, temperature, humidity, and nutrient levels, using AI algorithms. Users can control and monitor their pods through a mobile app, receiving real-time updates and tips for maximizing yield. The pods are equipped with solar panels to ensure energy efficiency and sustainability.\n\nAdditionally, the business model includes a subscription service for nutrient packs and seeds, as well as an online community platform wh

In [12]:
for message in logs:
  print(message.content)

**Business Idea: Smart Vertical Farming Pods (SVFP)**

**Description:**
Smart Vertical Farming Pods (SVFP) are modular, self-sustaining farming units designed for urban environments, utilizing advanced hydroponics and IoT technology. Each pod is a compact, stackable unit that can be installed in backyards, rooftops, or even inside homes, allowing individuals and small businesses to grow fresh produce year-round, regardless of climate conditions.

The SVFP features an integrated smart system that monitors and optimizes growth conditions, including light, temperature, humidity, and nutrient levels, using AI algorithms. Users can control and monitor their pods through a mobile app, receiving real-time updates and tips for maximizing yield. The pods are equipped with solar panels to ensure energy efficiency and sustainability.

Additionally, the business model includes a subscription service for nutrient packs and seeds, as well as an online community platform where users can share tips, r

In [13]:
analysis_prompt = PromptTemplate(
  template=(
    "Analyze the following business ideia: "
    "Idea: {idea}"
    "Identify 3 keys strengths and 3 potential weaknesses of the ideia."
  )
)

In [14]:
analysis_chain = analysis_prompt | llm | parse_and_log_chain

In [15]:
analysis_result = analysis_chain.invoke(idea_result["output"])

In [16]:
analysis_result["output"]

'### Strengths\n\n1. **Sustainability and Environmental Impact**: The Smart Vertical Farming Pods (SVFP) promote sustainable agriculture by utilizing hydroponics and solar energy. This reduces the carbon footprint associated with traditional farming and transportation of produce, making it an attractive option for environmentally conscious consumers.\n\n2. **Urban Adaptability**: The modular and compact design of the SVFP allows it to fit into various urban settings, such as rooftops, backyards, and even indoor spaces. This adaptability addresses the growing demand for local food production in urban areas, where space is often limited, and can help mitigate food deserts.\n\n3. **Technological Integration**: The use of IoT technology and AI algorithms to monitor and optimize growth conditions provides users with a high level of control and efficiency. The mobile app enhances user experience by offering real-time updates and personalized tips, making it easier for individuals, including 

In [17]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods (SVFP)**\n\n**Description:**\nSmart Vertical Farming Pods (SVFP) are modular, self-sustaining farming units designed for urban environments, utilizing advanced hydroponics and IoT technology. Each pod is a compact, stackable unit that can be installed in backyards, rooftops, or even inside homes, allowing individuals and small businesses to grow fresh produce year-round, regardless of climate conditions.\n\nThe SVFP features an integrated smart system that monitors and optimizes growth conditions, including light, temperature, humidity, and nutrient levels, using AI algorithms. Users can control and monitor their pods through a mobile app, receiving real-time updates and tips for maximizing yield. The pods are equipped with solar panels to ensure energy efficiency and sustainability.\n\nAdditionally, the business model includes a subscription service for nutrient packs and seeds, as well as an online community platform wh

Report Generation

In [18]:
report_prompt = PromptTemplate(
  template=(
    "Here is business analysis: "
    "Strengths & Weaknesses: {output}"
    "Generate a structured business report."
  )
)

In [19]:
class AnalysisReport(BaseModel):
  """Strenghts and Weaknesses about a business ideia"""
  strenghts: list = Field(default=[], description="Idea's strenghts list")
  weaknesses: list = Field(default=[], description="Idea's weaknesses list")

In [21]:
report_chain = (
  report_prompt | llm.with_structured_output(AnalysisReport, method="function_calling")
)

In [22]:
report_result = report_chain.invoke(analysis_result["output"])

In [23]:
report_result

AnalysisReport(strenghts=['Sustainability and Environmental Impact: The Smart Vertical Farming Pods (SVFP) promote sustainable agriculture by utilizing hydroponics and solar energy, reducing the carbon footprint associated with traditional farming and transportation of produce.', 'Urban Adaptability: The modular and compact design of the SVFP allows it to fit into various urban settings, addressing the growing demand for local food production in urban areas and helping to mitigate food deserts.', 'Technological Integration: The use of IoT technology and AI algorithms to monitor and optimize growth conditions provides users with a high level of control and efficiency, enhancing user experience through a mobile app that offers real-time updates and personalized tips.'], weaknesses=['Initial Cost and Accessibility: The upfront investment required for purchasing a Smart Vertical Farming Pod may be a barrier for some potential users, particularly in lower-income urban areas, potentially det

In [24]:
report_result.strenghts

['Sustainability and Environmental Impact: The Smart Vertical Farming Pods (SVFP) promote sustainable agriculture by utilizing hydroponics and solar energy, reducing the carbon footprint associated with traditional farming and transportation of produce.',
 'Urban Adaptability: The modular and compact design of the SVFP allows it to fit into various urban settings, addressing the growing demand for local food production in urban areas and helping to mitigate food deserts.',
 'Technological Integration: The use of IoT technology and AI algorithms to monitor and optimize growth conditions provides users with a high level of control and efficiency, enhancing user experience through a mobile app that offers real-time updates and personalized tips.']

In [25]:
report_result.weaknesses

['Initial Cost and Accessibility: The upfront investment required for purchasing a Smart Vertical Farming Pod may be a barrier for some potential users, particularly in lower-income urban areas, potentially deterring adoption despite a subscription model for ongoing support.',
 'Technical Complexity: Users who are not tech-savvy may find it challenging to operate the system effectively, and reliance on technology means that any technical failures or malfunctions could disrupt the growing process, leading to potential crop loss.',
 'Market Competition and Differentiation: The vertical farming and urban agriculture market is becoming increasingly competitive, requiring SVFP to clearly differentiate itself from existing products and services, which may necessitate significant marketing efforts and innovation.']

End to End Chain

In [ ]:
e2e_chain = (
  RunnablePassthrough()
  | idea_chain
  | RunnableParallel(idea=RunnablePassthrough())
  | analysis_chain
  | report_chain
)

In [29]:
e2e_result = e2e_chain.invoke("agro")

In [31]:
e2e_result.strenghts

['Sustainability and Local Food Production: The Smart Vertical Farming Pods (SVFP) promote sustainable agriculture by enabling local food production in urban areas. This reduces the carbon footprint associated with transporting food and encourages environmentally conscious practices among consumers.',
 'Technological Integration: The use of IoT technology and real-time data analysis allows for precise monitoring and optimization of plant growth. This can lead to higher yields and healthier crops, making the SVFPs appealing to both novice and experienced urban gardeners.',
 'Community Engagement: The community platform fosters a sense of connection among users, encouraging knowledge sharing, recipe exchanges, and even trading surplus produce. This social aspect can enhance customer loyalty and create a supportive network of urban farmers.']

In [32]:
e2e_result.weaknesses

['Initial Cost and Accessibility: The upfront investment for purchasing and installing SVFPs may be a barrier for some potential users, particularly in lower-income urban areas. This could limit the market reach and adoption of the product.',
 'Technical Complexity: While the technology aims to simplify the farming process, some users may find the IoT features and app interface challenging to navigate. This could deter less tech-savvy individuals from fully utilizing the pods or lead to frustration.',
 'Market Competition and Differentiation: The vertical farming and hydroponics market is becoming increasingly competitive, with various players offering similar solutions. SVFPs will need to clearly differentiate themselves through unique features, pricing strategies, or marketing approaches to capture and retain market share.']

In [33]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods (SVFP)**\n\n**Description:**\nSmart Vertical Farming Pods (SVFP) are modular, self-sustaining farming units designed for urban environments, utilizing advanced hydroponics and IoT technology. Each pod is a compact, stackable unit that can be installed in backyards, rooftops, or even inside homes, allowing individuals and small businesses to grow fresh produce year-round, regardless of climate conditions.\n\nThe SVFP features an integrated smart system that monitors and optimizes growth conditions, including light, temperature, humidity, and nutrient levels, using AI algorithms. Users can control and monitor their pods through a mobile app, receiving real-time updates and tips for maximizing yield. The pods are equipped with solar panels to ensure energy efficiency and sustainability.\n\nAdditionally, the business model includes a subscription service for nutrient packs and seeds, as well as an online community platform wh